In [1]:
import time
from time import perf_counter

x= perf_counter()
time.sleep(1)
y=perf_counter()
t=y-x
print(f'sum = {sum}, time taken = {t}')


sum = <built-in function sum>, time taken = 1.0005080999981146


In [2]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA

In [3]:
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

DB_FAISS_PATH = "vectorstores/db_faiss"
db= FAISS.load_local(DB_FAISS_PATH, embeddings)

C:\Users\nsdev\code\llama_proj\aienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load llm 

llm = CTransformers(
        model = "llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type = "llama",
        max_new_tokens= 512,
        temperature= 0.5
    )

In [5]:
custom_prompt_template = """
Use the following pieces of information to answer the user's question. 
If you do not know th answer, please just say that you do not know the answer. Do not try to make up the answer.
Context: {context}
Question: {question}
Only returns the helpful answer below and nothing else.
Helpful answer:
"""

qa_prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])

# retrieval qa chain
qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={'prompt':qa_prompt}
    )

In [7]:
query = "What is diabetes?"

a=perf_counter()
res = qa_chain.invoke(query)
b=perf_counter()
t=b-a
print(f'time taken {t}')

In [8]:
res

{'query': 'What is diabetes?',
 'result': 'Diabetes is a disease characterized by an inability to process sugars in the diet due to a decrease or total absence of insulin production.',
 'source_documents': [Document(page_content='Diabetes —A disease characterized by an inability\nto process sugars in the diet, due to a decrease in ortotal absence of insulin production. May requireinjections of insulin before meals to aid in themetabolism of sugars.\nDuodenum —The first section of the small intestine\nthat receives partly digested material from thestomach.Endocrine —A system of organs that produces\nchemicals that go into the bloodstream to reachother organs whose functioning they affect.', metadata={'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf', 'page': 186}),
  Document(page_content='Diabetic coma —Reduced level of consciousness\nthat requires immediate medical attention.horizon, and traveling after dark can also minimize\nsymptoms.\nFood should be fresh, properly p

In [9]:
answer = res['result']
sources = res['source_documents']

In [10]:
answer

'Diabetes is a disease characterized by an inability to process sugars in the diet due to a decrease or total absence of insulin production.'

In [11]:
sources

[Document(page_content='Diabetes —A disease characterized by an inability\nto process sugars in the diet, due to a decrease in ortotal absence of insulin production. May requireinjections of insulin before meals to aid in themetabolism of sugars.\nDuodenum —The first section of the small intestine\nthat receives partly digested material from thestomach.Endocrine —A system of organs that produces\nchemicals that go into the bloodstream to reachother organs whose functioning they affect.', metadata={'source': 'data\\Gale Encyclopedia of Medicine Vol. 4 (N-S).pdf', 'page': 186}),
 Document(page_content='Diabetic coma —Reduced level of consciousness\nthat requires immediate medical attention.horizon, and traveling after dark can also minimize\nsymptoms.\nFood should be fresh, properly prepared, and eaten\nslowly. Overeating, tight-fitting clothes, and strenuousactivity immediately after a meal should be avoided.\nResources\nBOOKS\nThe Doctors Book of Home Remedies. Emmaus, PA: Rodale\nPres